## Before we start:
* Final assignment available on EduFlow during Monday class.

### A few questions:
* **Other tools for visualization:**
  * [Power BI](https://powerbi.microsoft.com/)
  * [Looker Studio](https://lookerstudio.google.com/) (formerly Google Data Studio)
  * [Tableau](https://www.tableau.com/)
  * +++ A host of other solutions
* **Why Python & Pandas vs other more accessible visualization tools?**
  * In short: Python for scale, performance, flexability > SPEED.
  * Typically Python for EDA (Exploratory Data Analysis); Power BI etc for more general analysis / business reporting.
  * Combine them:
    * Python to get, filter, merge data (e.g. via API)
    * Initial analysis/QA using visualization (e.g. Pandas)
    * Generate output (csv/excel/json)
    * Load into Power BI/Tableau etc for further analysis and more advanced visualizations. 
* **Assignment 4.3**


In [ ]:
# Objective x.x

# 4.4 Visualizing Data in Python